### Model Deployment

In [ ]:
import pandas as pd

df = pd.read_csv("data/actual-data.csv")

#read the dataset
df

In [ ]:
# combined all columns into 1 to be tokenized and fetch into model
df['combined_text'] = df.apply(lambda row: f"Bank: {row['Bank']} Currency: {row['Currency']} Date: {row['Date']} Amount: {row['Amount']} Text: {row['Text']}", axis=1)

In [ ]:
df_test = df[['combined_text']]
df_test

In [ ]:
# export the processed df as a csv file, unfilter the code to process
df_test.to_csv('data/actual-data-pd-out.csv', index=False)

In [ ]:
# load the processed csv file with two column "input" & 'label' under dataset library
from datasets import load_dataset

dataset_test = load_dataset("csv", data_files='data/actual-data-pd-out.csv')

In [ ]:
dataset_test

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

# Load the model
model = AutoModelForSequenceClassification.from_pretrained("model/bank-classifier-model")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("model/bank-classifier-tokenizer")

In [ ]:
def tokenize_function(example):
    return tokenizer(example["combined_text"], truncation=True)


tokenized_datasets_test = dataset_test.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
tokenized_datasets_test

In [ ]:
# the model will not recognised column "combined_text", lets remove it
tokenized_datasets_test = tokenized_datasets_test.remove_columns(["combined_text"])

tokenized_datasets_test

In [ ]:
from transformers import Trainer, TrainingArguments
import torch

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
# Make predictions
predictions = trainer.predict(tokenized_datasets_test["train"])

In [ ]:
print(predictions)

In [ ]:
import numpy as np

# Extract predicted labels
preds = np.argmax(predictions.predictions, axis=-1)

# Display predicted labels
print(preds)

In [ ]:
# Define label mapping dictionary
label_mapping = {
    'BANK CHARGES': 0,
    'FOREX PURCHASE': 1,
    'FUND TRANSFER': 2,
    'INCOMING': 3,
    'INTEREST': 4,
    'LF TO ADVISE': 5,
    'OUTGOING': 6,
    'PETTY CASH': 7,
    'SALARY': 8,
    'SALARY ADVANCE': 9,
    'TAX': 10
}

In [ ]:
# Create inverse mapping
inverse_label_mapping = {v: k for k, v in label_mapping.items()}

In [ ]:
inverse_label_mapping

In [ ]:
# Convert predicted numerical labels to string labels
predicted_labels = [inverse_label_mapping[pred] for pred in preds]

In [ ]:
df['prediced_label'] = predicted_labels

In [ ]:
df

In [ ]:
# export the processed df as a csv file, unfilter the code to process
df.to_csv('data/actual-data-result.csv', index=False)